## RDF→CSV + standardisation CSV

In [32]:
import pandas as pd
from rdflib import Graph
import os

# =========================
# PATHS (tes fichiers)
# =========================
CTA_RDF_DAILY_PATH = r"C:\Fil Blanc\CTA Chicago - Ridership - Bus Routes - Daily Type Averages & Totals (RDF).rdf"
CTA_RDF_MONTHLY_PATH = r"C:\Fil Blanc\CTA Chicago - Ridership - Bus Routes - Monthly Day-Type Averages & Totals (RDF).rdf"

PHILLY_MODE_CSV_PATH = r"C:\Fil Blanc\Average_Daily_Ridership_By_Mode - City of Philadelphia.csv"
PHILLY_ROUTE_CSV_PATH = r"C:\Fil Blanc\Average_Daily_Ridership_By_Route - City of Philadelphia.csv"

# =========================
# 1) RDF -> DataFrame (CTA)
# =========================
def rdf_to_df(rdf_path: str) -> pd.DataFrame:
    g = Graph()
    # rdflib détecte le format automatiquement pour ces fichiers
    g.parse(rdf_path)

    rows = {}
    for s, p, o in g:
        p_str = str(p)

        # ignorer metadata RDF
        if p_str.endswith("rdf-syntax-ns#type") or p_str.endswith("/terms#rowID"):
            continue

        key = p_str.split("/")[-1]
        if "#" in key:
            key = key.split("#")[-1]

        rows.setdefault(str(s), {})[key] = str(o)

    return pd.DataFrame.from_dict(rows, orient="index").reset_index(drop=True)

# =========================
# 2) CTA Monthly -> table analytiques DayType
# =========================
def rdf_monthly_chicago_to_df(file_path):
    g = Graph()
    g.parse(file_path, format="xml")

    data = {}

    for s, p, o in g:
        s = str(s)
        p = str(p)
        o = str(o)

        if s not in data:
            data[s] = {
                "subject": s,
                "city": "Chicago"
            }

        field = p.split("/")[-1]

        if field in [
            "route",
            "routename",
            "month_beginning",
            "avg_weekday_rides",
            "avg_saturday_rides",
            "avg_sunday_holiday_rides",
            "monthtotal"
        ]:
            data[s][field] = o

    df = pd.DataFrame(data.values())

    df["month_beginning"] = pd.to_datetime(df["month_beginning"], errors="coerce")

    numeric_cols = [
        "avg_weekday_rides",
        "avg_saturday_rides",
        "avg_sunday_holiday_rides",
        "monthtotal"
    ]

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df

df_Monthly = rdf_monthly_chicago_to_df("CTA Chicago - Ridership - Bus Routes - Monthly Day-Type Averages & Totals (RDF).rdf")

# =========================
# 3) CTA Daily -> table journalière
# =========================
def build_cta_daily(rdf_path: str) -> pd.DataFrame:
    df = rdf_to_df(rdf_path)

    df["Date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
    df["Ridership"] = pd.to_numeric(df["rides"], errors="coerce")

    df = df.rename(columns={"route": "RouteID"})
    df["City"] = "Chicago"
    df["Mode"] = "Bus"

    # W/A/U -> labels
    day_map = {"W": "Weekday", "A": "Saturday", "U": "Sunday/Holiday"}
    df["DayType"] = df["daytype"].map(day_map).fillna(df["daytype"])

    out = df[["City","Mode","Date","RouteID","DayType","Ridership"]].copy()
    return out

# =========================
# 4) Philly By Mode -> standardisation
# =========================
def build_philly_monthly_by_mode(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # construire une date mensuelle standard (YYYY-MM-01)
    df["MonthBeginning"] = pd.to_datetime(
        df["Calendar_Year"].astype(str) + "-" +
        df["Calendar_Month"].astype(str).str.zfill(2) + "-01",
        errors="coerce"
    ).dt.date

    df["AverageDailyRidership"] = pd.to_numeric(df["Average_Daily_Ridership"], errors="coerce")

    out = pd.DataFrame({
        "City": "Philadelphia",
        "Mode": df["Mode"],
        "MonthBeginning": df["MonthBeginning"],
        "DayType": "All",
        "RouteID": "ALL",
        "RouteName": "ALL",
        "AverageDailyRidership": df["AverageDailyRidership"],
        "Source": df.get("Source", pd.Series([""] * len(df)))
    })
    return out

# =========================
# 5) Philly By Route -> standardisation
# =========================
def build_philly_monthly_by_route(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    df["MonthBeginning"] = pd.to_datetime(
        df["Calendar_Year"].astype(str) + "-" +
        df["Calendar_Month"].astype(str).str.zfill(2) + "-01",
        errors="coerce"
    ).dt.date

    df["AverageDailyRidership"] = pd.to_numeric(df["Average_Daily_Ridership"], errors="coerce")

    # Ici, le CSV "by route" n’a pas Mode => on met "Bus" par défaut
    out = pd.DataFrame({
        "City": "Philadelphia",
        "Mode": "Bus",
        "MonthBeginning": df["MonthBeginning"],
        "DayType": "All",
        "RouteID": df["Route"].astype(str),
        "RouteName": df["Route"].astype(str),
        "AverageDailyRidership": df["AverageDailyRidership"],
        "Source": df.get("Source", pd.Series([""] * len(df)))
    })
    return out

# =========================
# RUN + EXPORT
# =========================
cta_monthly = build_cta_monthly_by_daytype(CTA_RDF_MONTHLY_PATH)
cta_daily = build_cta_daily(CTA_RDF_DAILY_PATH)
philly_mode = build_philly_monthly_by_mode(PHILLY_MODE_CSV_PATH)
philly_route = build_philly_monthly_by_route(PHILLY_ROUTE_CSV_PATH)

# Afficher tailles (vérification)
print("CTA monthly:", df_Monthly.shape)
print("CTA daily:", cta_daily.shape)
print("Philly mode:", philly_mode.shape)
print("Philly route:", philly_route.shape)

# Export CSV


OUTPUT_DIR = "data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df_Monthly.to_csv("data/chicago_cta_monthly_by_daytype.csv", index=False)
cta_daily.to_csv("data/chicago_cta_daily.csv", index=False)
philly_mode.to_csv("data/philadelphia_monthly_by_mode.csv", index=False)
philly_route.to_csv("data/philadelphia_monthly_by_route.csv", index=False)

print("✅ Fichiers créés dans data/")

CTA monthly: (500, 9)
CTA daily: (500, 6)
Philly mode: (492, 8)
Philly route: (10994, 8)
✅ Fichiers créés dans data/


## CTA Bus Tracker (Chicago) — template

In [21]:
import requests
import pandas as pd
from datetime import datetime, timezone
import os
import time

CTA_API_KEY = "D8yuKxkdzXiULT4ZxyKPZL5gm"
BASE = "https://www.ctabustracker.com/bustime/api/v3/"

def cta_call(endpoint, **params):
    params["key"] = CTA_API_KEY
    params["format"] = "json"
    r = requests.get(BASE + endpoint, params=params, timeout=30)
    r.raise_for_status()
    data = r.json().get("bustime-response", {})
    if "error" in data and data["error"]:
        msg = data["error"][0].get("msg", str(data["error"][0]))
        raise ValueError(msg)
    return data

def cta_routes():
    data = cta_call("getroutes")
    routes = data.get("routes", [])
    if not routes:
        raise ValueError("Aucune route retournée.")
    df = pd.json_normalize(routes)

    # Sécuriser la colonne rt
    df["rt"] = df["rt"].astype(str).str.strip()
    df = df[df["rt"].notna() & (df["rt"] != "")]

    return df

def cta_vehicles_safe():
    routes_df = cta_routes()
    route_ids = routes_df["rt"].tolist()

    frames = []
    errors = []
    snap = datetime.now(timezone.utc)

    for rt in route_ids:
        try:
            data = cta_call("getvehicles", rt=rt)
            veh = data.get("vehicle", [])
            if veh:
                df = pd.json_normalize(veh)
                df["snapshot_ts"] = snap
                df["City"] = "Chicago"
                frames.append(df)
        except Exception as e:
            errors.append({"route": rt, "error": str(e)})

        # petite pause pour éviter rate-limit
        time.sleep(0.1)

    if not frames:
        raise ValueError("Aucun véhicule récupéré sur toutes les routes (très rare).")

    vehicles_df = pd.concat(frames, ignore_index=True)
    errors_df = pd.DataFrame(errors)

    return vehicles_df, errors_df

# ===== RUN =====
cta_df, cta_errors = cta_vehicles_safe()
print("CTA vehicles:", cta_df.shape)
print("CTA routes with errors:", cta_errors.shape)

# ===== EXPORT =====
os.makedirs("data", exist_ok=True)
cta_df.to_csv("data/cta_realtime_vehicles.csv", index=False)
cta_errors.to_csv("data/cta_realtime_errors.csv", index=False)

print("✅ data/cta_realtime_vehicles.csv créé")
print("✅ data/cta_realtime_errors.csv créé (routes problématiques)")


CTA vehicles: (216, 20)
CTA routes with errors: (59, 2)
✅ data/cta_realtime_vehicles.csv créé
✅ data/cta_realtime_errors.csv créé (routes problématiques)


## SEPTA TransitView (Philadelphia) — template

In [19]:
import requests, pandas as pd
from datetime import datetime, timezone
import os

SEPTA_URL = "https://www3.septa.org/api/TransitViewAll/"

def fetch_septa_vehicles():
    r = requests.get(SEPTA_URL, timeout=30)
    r.raise_for_status()
    data = r.json()

    rows = []
    for mode, vehicles in data.items():
        if isinstance(vehicles, list):
            for v in vehicles:
                v["mode"] = mode
                rows.append(v)

    if not rows:
        raise ValueError("⚠️ AUCUN véhicule retourné par l’API SEPTA")

    df = pd.json_normalize(rows)
    df["snapshot_ts"] = datetime.now(timezone.utc)
    df["City"] = "Philadelphia"
    return df

septa_df = fetch_septa_vehicles()
print("SEPTA vehicles:", septa_df.shape)

os.makedirs("data", exist_ok=True)
septa_df.to_csv("data/septa_realtime_vehicles.csv", index=False)
print("✅ data/septa_realtime_vehicles.csv créé")


SEPTA vehicles: (1, 87)
✅ data/septa_realtime_vehicles.csv créé


## Néttoyage 

In [41]:
import pandas as pd
import os

# =========================
# PATHS (fichiers uploadés)
# =========================
FILES = {
    "chi_daily": "C:\\Fil Blanc\\data\\chicago_cta_daily.csv",
    "chi_monthly": "C:\\Fil Blanc\\data\\chicago_cta_monthly_by_daytype.csv",
    "phl_route": "C:\\Fil Blanc\\data\\philadelphia_monthly_by_route.csv",
    "phl_mode": "C:\\Fil Blanc\\data\\philadelphia_monthly_by_mode.csv",
    "cta_rt": "C:\\Fil Blanc\\data\\cta_realtime_vehicles.csv",
    "septa_rt": "C:\\Fil Blanc\\data\\septa_realtime_vehicles.csv",
}

OUT_DIR = "C:\\Fil Blanc\\data\\cleaned"
os.makedirs(OUT_DIR, exist_ok=True)

# =========================
# HELPERS
# =========================
def clean_str(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()
            df.loc[df[c].isin(["", "nan", "None"]), c] = pd.NA
    return df

def clean_num(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)
    return df

# =========================
# 1) CHICAGO DAILY
# =========================
chi_daily = pd.read_csv(FILES["chi_daily"])
chi_daily["Date"] = pd.to_datetime(chi_daily["Date"], errors="coerce")
chi_daily = clean_str(chi_daily, ["RouteID", "DayType", "City", "Mode"])
chi_daily = clean_num(chi_daily, ["Ridership"])
chi_daily = chi_daily.dropna(subset=["Date", "RouteID"])
chi_daily = chi_daily.drop_duplicates(subset=["Date", "RouteID", "DayType"])

# =========================
# 2) CHICAGO MONTHLY BY DAYTYPE
# =========================
chi_monthly = pd.read_csv(FILES["chi_monthly"])
# 1) supprimer subject
if "subject" in chi_monthly.columns:
    chi_monthly = chi_monthly.drop(columns=["subject"])

# 2) nettoyer noms (optionnel mais propre)
chi_monthly.columns = (
    chi_monthly.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

# 3) conversions types
chi_monthly["month_beginning"] = pd.to_datetime(chi_monthly["month_beginning"], errors="coerce")

# route = identifiant de ligne
chi_monthly["route"] = chi_monthly["route"].astype(str).str.strip()
chi_monthly["routename"] = chi_monthly["routename"].astype(str).str.strip()
chi_monthly["city"] = chi_monthly["city"].astype(str).str.strip()

# numériques (rides + total)
num_cols = [
    "monthtotal",
    "avg_weekday_rides",
    "avg_saturday_rides",
    "avg_sunday_holiday_rides"
]
for c in num_cols:
    if c in chi_monthly.columns:
        chi_monthly[c] = pd.to_numeric(chi_monthly[c], errors="coerce").fillna(0)

# 4) supprimer lignes invalides
chi_monthly = chi_monthly.dropna(subset=["month_beginning"])
chi_monthly = chi_monthly[chi_monthly["route"].notna() & (chi_monthly["route"] != "")]

# 5) supprimer doublons (clé correcte pour ce dataset)
chi_monthly = chi_monthly.drop_duplicates(subset=["month_beginning", "route"], keep="last")
# =========================
# 3) PHILADELPHIA MONTHLY BY ROUTE
# =========================
phl_route = pd.read_csv(FILES["phl_route"])
phl_route["MonthBeginning"] = pd.to_datetime(phl_route["MonthBeginning"], errors="coerce")
phl_route = clean_str(phl_route, ["RouteID", "RouteName", "City", "Mode"])
phl_route = clean_num(phl_route, ["AverageDailyRidership"])
phl_route = phl_route.dropna(subset=["MonthBeginning", "RouteID"])
phl_route = phl_route.drop_duplicates(subset=["MonthBeginning", "RouteID"])

# =========================
# 4) PHILADELPHIA MONTHLY BY MODE
# =========================
phl_mode = pd.read_csv(FILES["phl_mode"])
phl_mode["MonthBeginning"] = pd.to_datetime(phl_mode["MonthBeginning"], errors="coerce")
phl_mode = clean_str(phl_mode, ["Mode", "City"])
phl_mode = clean_num(phl_mode, ["AverageDailyRidership"])
phl_mode = phl_mode.dropna(subset=["MonthBeginning", "Mode"])
phl_mode = phl_mode.drop_duplicates(subset=["MonthBeginning", "Mode"])

# =========================
# 5) CTA REALTIME
# =========================
cta_rt = pd.read_csv(FILES["cta_rt"])
if "snapshot_ts" in cta_rt.columns:
    cta_rt["snapshot_ts"] = pd.to_datetime(cta_rt["snapshot_ts"], errors="coerce", utc=True)
cta_rt = clean_str(cta_rt, ["vid", "rt"])
cta_rt = cta_rt.dropna(subset=["vid", "rt"])
cta_rt = cta_rt.drop_duplicates(subset=["snapshot_ts", "vid"])

# =========================
# 6) SEPTA REALTIME
# =========================
septa_rt = pd.read_csv(FILES["septa_rt"])

# 1) NORMALISER TOUS LES NOMS DE COLONNES
# (supprime espaces, casse, tirets, etc.)
septa_rt.columns = (
    septa_rt.columns
    .str.strip()
    .str.replace(" ", "")
    .str.replace("-", "")
    .str.replace("_", "")
    .str.lower()
)

print("Colonnes normalisées :", list(septa_rt.columns))

# 2) DÉTECTER LA COLONNE ID VÉHICULE
# après normalisation, on cherche "vehicleid"
if "vehicleid" not in septa_rt.columns:
    raise ValueError(
        " Colonne VehicleId introuvable après normalisation. "
        f"Colonnes disponibles : {list(septa_rt.columns)}"
    )

# 3) RENOMMER PROPREMENT EN VehicleID
septa_rt = septa_rt.rename(columns={"vehicleid": "VehicleID"})

# 4) GÉRER snapshot_ts (s'il existe)
if "snapshotts" in septa_rt.columns:
    septa_rt = septa_rt.rename(columns={"snapshotts": "snapshot_ts"})
    septa_rt["snapshot_ts"] = pd.to_datetime(
        septa_rt["snapshot_ts"], errors="coerce", utc=True
    )

# 5) NETTOYER VehicleID
septa_rt["VehicleID"] = septa_rt["VehicleID"].astype(str).str.strip()
septa_rt.loc[
    septa_rt["VehicleID"].isin(["", "nan", "none"]),
    "VehicleID"
] = pd.NA

# 6) SUPPRIMER LIGNES INVALIDES
septa_rt = septa_rt.dropna(subset=["VehicleID"])

# 7) SUPPRIMER DOUBLONS
if "snapshot_ts" in septa_rt.columns:
    septa_rt = septa_rt.drop_duplicates(subset=["snapshot_ts", "VehicleID"])
else:
    septa_rt = septa_rt.drop_duplicates(subset=["VehicleID"])


# =========================
# EXPORT
# =========================
chi_daily.to_csv(f"{OUT_DIR}/chicago_cta_daily_clean.csv", index=False)
chi_monthly.to_csv(f"{OUT_DIR}/chicago_cta_monthly_by_daytype_clean.csv", index=False)
phl_route.to_csv(f"{OUT_DIR}/philadelphia_monthly_by_route_clean.csv", index=False)
phl_mode.to_csv(f"{OUT_DIR}/philadelphia_monthly_by_mode_clean.csv", index=False)
cta_rt.to_csv(f"{OUT_DIR}/cta_realtime_vehicles_clean.csv", index=False)
septa_rt.to_csv(f"{OUT_DIR}/septa_realtime_vehicles_clean.csv", index=False)

print("✅ Nettoyage terminé")
print("Chicago daily:", chi_daily.shape)
print("Chicago monthly:", chi_monthly.shape)
print("Philly by route:", phl_route.shape)
print("Philly by mode:", phl_mode.shape)
print("CTA realtime:", cta_rt.shape)
print("SEPTA realtime:", septa_rt.shape)


Colonnes normalisées : ['108', 't4', 't1', '103', '51', '47', '17', '53', '37', '33', '117', 'l1owl', '23', '29', '52', 'blvddir', 'g1', '20', '25', 't5', '109', '42', '125', '63', '9', '18', '66', '93', '119', '57', '12', '14', '45', '54', '60', '65', 'd1', 'l1', 'm1', '55', '115', '104', '114', '82', 'd2', '96', 't5bus', '4', '58', '41', 'b1owl', '61', '5', '48', '56', '139', '6', '79', '124', '75', '7', '113', 'k', '22', 't2', '99', '132', '67', 't3', '73', '2', '31', '16', '59', '123', '30', '39', '3', '46', 'b1', '27', '38', '49', '68', 'mode', 'snapshotts', 'city']


ValueError:  Colonne VehicleId introuvable après normalisation. Colonnes disponibles : ['108', 't4', 't1', '103', '51', '47', '17', '53', '37', '33', '117', 'l1owl', '23', '29', '52', 'blvddir', 'g1', '20', '25', 't5', '109', '42', '125', '63', '9', '18', '66', '93', '119', '57', '12', '14', '45', '54', '60', '65', 'd1', 'l1', 'm1', '55', '115', '104', '114', '82', 'd2', '96', 't5bus', '4', '58', '41', 'b1owl', '61', '5', '48', '56', '139', '6', '79', '124', '75', '7', '113', 'k', '22', 't2', '99', '132', '67', 't3', '73', '2', '31', '16', '59', '123', '30', '39', '3', '46', 'b1', '27', '38', '49', '68', 'mode', 'snapshotts', 'city']